# GPU-Accelerated Tree SHAP on AWS

With the release of XGBoost 1.3 comes an exciting new feature for model interpretability — GPU accelerated SHAP values. SHAP values are a technique for local explainability of model predictions. That is, they give you the ability to examine the impact of various features on model output in a principled way. SHAP at its core describes the average impact from adding a feature to a model, but does it in a way that attempts to account for all possible subsets of the other features as well. See [GPU-Accelerated SHAP values with XGBoost 1.3 and RAPIDS](https://medium.com/rapids-ai/gpu-accelerated-shap-values-with-xgboost-1-3-and-rapids-587fad6822) for more details.

In this notebook, we provide an example of training an XGBoost model with AWS SageMaker's [XGBoost estimator](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html), and then use SHAP values to identify key features and feature interactions in our dataset. SHAP values have been available in XGBoost for several versions already, but 1.3 brings GPU acceleration, reducing computation time by up to 20x for SHAP values and 340x for SHAP interaction values. This is powered under the hood by RAPIDS GPUTreeShap, which offers portable CUDA C++ implementations of SHAP algorithms for decision tree models.

We will be using the NYC Taxi dataset, which captures yellow cab trip details in New York in January 2020, stored in CSV format without any compression. The machine learning objective with this dataset is to predict whether a trip had an above average tip (>$2.20).

We also provide functionality with the California housing dataset. This is a famous dataset of house prices and attributes in California from the 1990 Census, available via [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html). In order to use this dataset, the user must adjust (comment/uncomment) sections of the code accordingly. 

## 0. Preliminary Setup

This notebook was tested in an Amazon [SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html) notebook, on a ml.t3.medium instance with Python 3 (Data Science) kernel. As a preliminary step, we first ensure that the latest version of SageMaker is installed: 

In [ ]:
!pip install -U sagemaker

Let's start by specifying:
1. The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
2. The IAM role arn used to give training and hosting access to your data. See the [AWS documentation](https://docs.aws.amazon.com/glue/latest/dg/create-an-iam-role-sagemaker-notebook.html) regarding these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regex with a the appropriate full IAM role arn string(s).

In [ ]:
import io
import os
import boto3
import sagemaker
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-xgboost-inference-script-mode"

## 1. Training the XGBoost model

SageMaker can now run an XGboost script using the [XGBoost estimator](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html). A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. In this notebook, we use the training script [train.py](train.py).

After setting training parameters, we kick off training. 

To run our training script on SageMaker, we construct a `sagemaker.xgboost.estimator.XGBoost` estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __hyperparameters__ *(optional)*: A dictionary passed to the train function as hyperparameters. For the XGBoost estimator, the list of possible hyperparameters can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html). By default, within our training script we have set "tree_method" to "gpu_hist" and "predictor" to "gpu_predictor" to enable GPU-accelerated training and SHAP values. For the CPU version, set "tree_method" to "hist" and "predictor" to "cpu_predictor."
* __role__: Role ARN
* __instance_type__: The type of SageMaker instance to be used. A list of instance types can be found [here](https://aws.amazon.com/ec2/instance-types/). The `ml.g4dn.xlarge` is a GPU instance; other GPUs can be specified, or a CPU instance type for the CPU run.
* __framework_version__: SageMaker XGBoost version you want to use for executing your model training code, e.g., `0.90-1`, `0.90-2`, `1.0-1`, or `1.3-1`. We must use `1.3-1` for GPU accelerated SHAP values.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.

In [ ]:
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

job_name = "DEMO-xgboost-inference-script-mode-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("Training job", job_name)

hyperparameters = {
    "max_depth": "6",
    "eta": "0.3",
    "gamma": "0",
    "min_child_weight": "1",
    "subsample": "1",
    "objective": "reg:squarederror",
    "num_round": "500",
    "verbosity": "1",
    # "tree_method": "hist", "predictor": "cpu_predictor",  # for CPU version
    
    # dataset-specific params
    # "sklearn_dataset": "sklearn.datasets.fetch_california_housing()",  # uncomment to use California housing dataset
    "content_type": "csv",  # comment out when using California housing dataset
    "label_column": "17",   # comment out when using California housing dataset
}

instance_type = "ml.g4dn.xlarge"  # "ml.c5.xlarge" for CPU, "ml.g4dn.xlarge" for GPU

xgb_script_mode_estimator = XGBoost(
    entry_point="train.py",
    hyperparameters=hyperparameters,
    role=role,
    instance_count=1,
    instance_type=instance_type,
    framework_version="1.3-1",
    output_path="s3://{}/{}/{}/output".format(bucket, prefix, job_name),
)

In [ ]:
"""
Since the estimator requires a valid file type but we are specifying a sklearn_dataset, 
we pass in a path to a tiny csv file which will not be used.
"""
content_type = "text/csv"  # MIME type
train_input = TrainingInput(
    "s3://sagemaker-rapids-hpo-us-east-1/dummy_data.csv", content_type=content_type
)


# Example of using a public CSV dataset - remember to remove "sklearn_dataset" hyperparameter
# Comment out when using California housing dataset
train_input = TrainingInput(
    "s3://sagemaker-rapids-hpo-us-east-1/NYC_taxi/NYC_taxi_tripdata_2020-01.csv", content_type="text/csv"
)

### 1.1 Train XGBoost Estimator on California Housing Data

Training is as simple as calling `fit` on the Estimator. This will start a SageMaker Training job that will download the data, invoke the entry point code (in the provided script file), and save any model artifacts that the script creates. 

Note in the cell above that we can specify "sklearn_dataset": "sklearn.datasets.fetch_california_housing()" as a parameter in order to use the California housing dataset from scikit-learn. However, calling `fit` on the Estimator requires a valid filepath, but we do not want to have to download the scikit-learn data. Instead, we refer to a small (only a couple of bytes large) existing dummy CSV file, which is immediately discarded by `train.py` upon receiving a valid "sklearn_dataset" input.

In [ ]:
%%time
xgb_script_mode_estimator.fit({"train": train_input}, job_name=job_name)

## 2. Deploying the XGBoost endpoint

After training, we can host the newly created model in SageMaker, and create an Amazon SageMaker endpoint – a hosted and managed prediction service that we can use to perform inference. If you call `deploy` after you call `fit` on an XGBoost estimator, it will create a SageMaker endpoint using the training script (i.e., `entry_point`). 

You can optionally specify other functions to customize the behavior of deserialization of the input request (`input_fn()`), serialization of the predictions (`output_fn()`), and how predictions are made (`predict_fn()`). First `input_fn()` is called, then its output is fed into `predict_fn()`, and finally `output_fn()` returns the predictions. Rather than defining three separate functions, you can also combine them all into a function `transform_fn()`, which is what we did for this example in **inference.py**. If any of these functions are not specified, the endpoint will use the default functions in the SageMaker XGBoost container. See the [SageMaker Python SDK documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/using_xgboost.html#sagemaker-xgboost-model-server) for details.

In this notebook, we will run a separate inference script and customize the endpoint to return [SHAP](https://github.com/slundberg/shap) values and interactions in addition to predictions. The inference script that we will run in this notebook is provided as the accompanying file `inference.py`. 

### 2.1 Deploy to an endpoint

Since the inference script is separate from the training script, here we use `XGBoostModel` to create a model from s3 artifacts and specify `inference.py` as the `entry_point`.

In [ ]:
from sagemaker.xgboost.model import XGBoostModel

model_data = xgb_script_mode_estimator.model_data
print(model_data)

xgb_inference_model = XGBoostModel(
    model_data=model_data,
    role=role,
    entry_point="inference.py",
    framework_version="1.3-1",
)

In [ ]:
predictor = xgb_inference_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=None, deserializer=None,
)

### 2.2 Make predictions

Now that we have fetched the dataset and trained an XGBoost regression model with 500 trees (using GPU acceleration), we can generate predictions on the training set. 

In [ ]:
print(predictor.serializer)
predictor.serializer = sagemaker.serializers.CSVSerializer()  # for NYC_taxi predictions. Comment out for sklearn predictions

In order to calculate predictions for the NYC Taxi dataset, we must read in the CSV file itself. We make predictions on 20,000 rows of the data. The following two cells may be skipped if you are generating predictions for the California housing dataset instead.

In [ ]:
import pandas as pd

data = pd.read_csv('s3://sagemaker-rapids-hpo-us-east-1/NYC_taxi/NYC_taxi_tripdata_2020-01.csv')
X = data.iloc[:,:-1]

In [ ]:
cutoff = 0

input_data = []
for _, row in X.iterrows():
    cutoff += 1
    if cutoff > 20000:
        break
    
    to_predict = []
    for i in range(row.shape[0]):
        to_predict.append(row[i])
        
    input_data.append(to_predict)

Here we make our predictions: 

In [ ]:
# input_data = "sklearn.datasets.fetch_california_housing()"  # uncomment to make predictions on California housing dataset
predictor_input = str(input_data) + ", predict"
predictions = predictor.predict(predictor_input)

Because the output is a string, we define a method to clean it up and re-cast it as a NumPy array.

In [ ]:
import numpy as np

def clean_array(arr, three_dim=False):
    cleaned_list = []
    arr_count = 0
    for num in arr:
        if '[' in num:
            arr_count += 1
            num = num.replace('[', '')
            cleaned_list.append(float(num))
        elif ']' in num:
            num = num.replace(']', '')
            cleaned_list.append(float(num))
        else: 
            cleaned_list.append(float(num))
            
    array = np.array(cleaned_list, dtype='float32')
    if three_dim:  # shap_interactions will be 3D
        y = int( len(array) / arr_count )
        x = int( arr_count / y )
        array = array.reshape(x, y, y)
    elif(arr_count > 1):
        y = int( len(array) / arr_count )
        array = array.reshape(arr_count, y)
    return array
    
    
predictions = clean_array(predictions[0])

### 2.3 Generate SHAP values

Here we generate the SHAP values of the training set. The time it took to compute all the SHAP values was: 

In [ ]:
predictor_input = str(input_data) + ", pred_contribs"

start = time.time()
shap_values = predictor.predict(predictor_input)
print("SHAP time {}".format(time.time() - start))

shap_values = clean_array(shap_values[0])

`shap_values` now contains a matrix where each row is a training instance from X and the columns contain the feature attributions (i.e. the amount that each feature contributed to the prediction). The last column in the output shap_values contains the ‘bias’ or the expected output of the model if no features were used. Each row always adds up exactly to the model prediction — this is a unique advantage of SHAP values compared to other model explanation techniques.

Model predictions can be inspected individually using this output, or we can aggregate the SHAP values to gain insight into global feature importance. Here we take the mean absolute contribution of each feature and plot their magnitude.

### 2.4 Compute SHAP interactions

Now with GPUTreeShap we can compute these interaction effects in a matter of seconds, even for large datasets with many features.

In [ ]:
predictor_input = str(input_data) + ", pred_interactions"

start = time.time()
shap_interactions = predictor.predict(predictor_input)
print("SHAP interactions time {}".format(time.time() - start))

shap_interactions = clean_array(shap_interactions[0], three_dim=True)

The NYC Taxi and California housing datasets are relatively small, with 17x17 and 8x8 possible feature interactions, respectively. For larger datasets, as shown in our paper, GPUTreeShap can reduce feature interaction computations from days to a matter of minutes.

The output `shap_interactions` contains a symmetric matrix of interaction terms for each row, where the element-wise sum evaluates to the model prediction. The diagonal terms represent the main effects for each feature or the impact of that feature excluding second-order interactions.

As before we can aggregate interactions to examine the most significant effects over the training set.

### 2.5 Delete the Endpoint

If you're done with this exercise, please run the `delete_endpoint` line in the cell below.  This will remove the hosted endpoint and avoid any charges from a stray instance being left on.

In [ ]:
predictor.delete_endpoint()

## 3. References

- [GPU-Accelerated SHAP values with XGBoost 1.3 and RAPIDS](https://medium.com/rapids-ai/gpu-accelerated-shap-values-with-xgboost-1-3-and-rapids-587fad6822)

- [SageMaker XGBoost Abalone example](https://github.com/aws/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/xgboost_abalone)

- [SageMaker XGBoost docs](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/index.html)